In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - OpenCLIP

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_open_clip.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_open_clip.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_open_clip.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview

This notebook demonstrates finetuning [OpenCLIP](https://github.com/mlfoundations/open_clip) with [CC3M](https://ai.google.com/research/ConceptualCaptions/download) dataset and deploying it on Vertex AI for online prediction.

### Objective

- Finetune the OpenCLIP model with [Vertex AI custom training](https://cloud.google.com/vertex-ai/docs/training/overview).
- Upload the model to [Vertex AI Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction).
- Deploy the model to a [Vertex AI Endpoint resource](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints).
- Run online predictions for zero-shot image classification.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Setup environment

**NOTE**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

### Colab only

In [ ]:
if "google.colab" in str(get_ipython()):
    ! pip3 install --upgrade google-cloud-aiplatform
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

    # Restart the notebook kernel after installs.
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Setup Google Cloud project

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).

1. [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs.

1. [Create a service account](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console) with `Vertex AI User`,  `Storage Object Admin`, and `GCS Storage Bucket Owner roles` roles for deploying fine tuned model to Vertex AI endpoint.

Fill the following variables for experiments environment:

In [ ]:
# Cloud project id.
PROJECT_ID = ""  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = ""  # @param {type:"string"}

# The GCS bucket for storing experiments output. Fill it without the 'gs://' prefix.
GCS_BUCKET = ""  # @param {type:"string"}

# The service account for deploying fine tuned model.
# The service account looks like:
# '<account_name>@<project>.iam.gserviceaccount.com'
# Follow step 5 above to create this account.
SERVICE_ACCOUNT = ""  # @param {type:"string"}

### Download data to Google Cloud Storage

In [ ]:
# Install the library for downloading training data.
!pip install img2dataset==1.41.0

In [ ]:
import os

!gcloud storage cp gs://gcc-data/Validation/GCC-1.1.0-Validation.tsv ./data.tsv  # Download list of URLs.
!sed -i '1s/^/caption	url
/' data.tsv  # Add column name.

In [ ]:
# Download images from URLs. It takes around 2min.
output_folder = "data"
!img2dataset --url_list data.tsv --input_format "tsv"\
    --output_folder {output_folder}\
    --url_col "url" --caption_col "caption" --output_format webdataset\
    --processes_count {os.cpu_count()} --thread_count {os.cpu_count()*4}\
    --image_size 256

In [ ]:
GCS_data_dir = f"{GCS_BUCKET}/CC3M-val-wds"

In [ ]:
# Upload data to GCS.
!gcloud storage cp -r data gs://{GCS_data_dir}

In [ ]:
# Get total number of samples, which is required by OpenCLIP training.
import json

n_samples = 0
for filename in [f for f in os.listdir(output_folder) if f.endswith("_stats.json")]:
    with open(os.path.join(output_folder, filename)) as f:
        n_samples += json.load(f)["successes"]
print(n_samples)

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)

### Define constants

In [ ]:
# The pre-built training docker image. It contains training scripts and models.
TRAIN_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-open-clip-train"

# The pre-built serving docker image. It contains serving scripts and models.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-open-clip-serve"

### Define common functions for job operations and test data preparations

In [ ]:
import base64
from datetime import datetime
from io import BytesIO

import matplotlib.pyplot as plt
import requests
from PIL import Image


def create_job_name(prefix):
    """Create a job name string with a prefix."""
    user = os.environ.get("USER")
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    job_name = f"{prefix}-{user}-{now}"
    return job_name


def deploy_model(checkpoint, model, precision, task):
    """Deploy a model to Vertex AI endpoint."""
    model_name = "openclip"
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-{task}-endpoint")
    serving_env = {
        "CHECKPOINT": checkpoint,
        "MODEL_ID": model,
        "PRECISION": precision,
        "TASK": task,
    }
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/transformers_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type="n1-standard-4",
        accelerator_type="NVIDIA_TESLA_V100",
        accelerator_count=1,
        deploy_request_timeout=1800,
        service_account=SERVICE_ACCOUNT,
    )
    return model, endpoint


def download_image(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content)).convert("RGB")


def image_to_base64(image):
    buffer = BytesIO()
    image.save(buffer, format="PNG")
    image_str = base64.b64encode(buffer.getvalue()).decode("utf-8")
    return image_str


def plot_images(images, rows=1, cols=None):
    fig, axes = plt.subplots(rows, cols if cols is not None else len(images))
    for ax, img in zip(axes, images):
        ax.imshow(img)
        ax.axis("off")

## Fine tune the model

This section fine tunes the OpenCLIP model with [CC3M](https://ai.google.com/research/ConceptualCaptions/download) dataset validation split for the image-text pre-training. It loads the pretrained checkpoint by OpenAI. You use a small model of ***RN50*** here; check [this list](https://github.com/mlfoundations/open_clip#pretrained-model-interface) for other options supported by OpenAI checkpoints, such as ***RN50, ViT-B-32, etc***.

One `n1-standard-4` machine with 1 `NVIDIA_TESLA_V100` is required to run the fine-tuning job. The fine-tuning job takes about 3min to complete training for 2 epochs.

The fine-tuned model will be saved after the job finishs and it can then be loaded for inference.

In [ ]:
machine_type = "n1-standard-4"
gpu_type = "NVIDIA_TESLA_V100"
num_gpus = 1

job_name = create_job_name("openclip")

model_name = "RN50"
precision = "amp"

job = aiplatform.CustomContainerTrainingJob(
    display_name=job_name, container_uri=TRAIN_DOCKER_URI, command=["torchrun"]
)

job.run(
    args=[
        f"--nproc_per_node={num_gpus}",
        "-m",
        "training.main",
        f"--name={job_name}",
        f"--logs=/gcs/{GCS_BUCKET}",
        f"--train-data=/gcs/{GCS_data_dir}/{{00000..00001}}.tar",
        f"--train-num-samples={n_samples}",
        "--dataset-type=webdataset",
        "--batch-size=32",
        "--precision=amp",
        "--workers=8",
        "--dataset-resampled",
        "--save-frequency=2",
        "--epochs=2",
        f"--model={model_name}",
        "--pretrained=openai",
        "--save-most-recent",
    ],
    boot_disk_size_gb=600,
    replica_count=1,
    machine_type=machine_type,
    accelerator_type=gpu_type,
    accelerator_count=num_gpus,
)

## Upload and Deploy models

This section uploads the fine-tuned model to Model Registry and deploys it on the Endpoint.

One `n1-standard-4` machine with 1 `NVIDIA_TESLA_V100` is required to deploy OpenCLIP model.

The model deployment step will take ~20 minutes to complete.

In [ ]:
# Prepare image samples
img_diagram = download_image(
    "https://raw.githubusercontent.com/mlfoundations/open_clip/main/docs/CLIP.png"
)
img_cat = download_image(
    "https://upload.wikimedia.org/wikipedia/commons/thumb/9/9d/Calicocats2222.jpg/220px-Calicocats2222.jpg"
)
plot_images([img_diagram, img_cat])

#### Zero-shot image classification

In [ ]:
model, endpoint = deploy_model(
    checkpoint=f"gs://{GCS_BUCKET}/{job_name}/checkpoints/epoch_latest.pt",
    model=model_name,
    precision=precision,
    task="zero-shot-image-classification",
)

NOTE: The model weights are downloaded after the deployment succeeds. An additional 5 minutes of waiting time is needed **after** the above model deployment step succeeds and before you run the next step below. Otherwise you might see a `ServiceUnavailable: 503 502:Bad Gateway` error when you send requests to the endpoint.

In [ ]:
instances = [
    {
        "text": ["a diagram", "a dog", "a cat"],
        "image": image_to_base64(img_diagram),
    },
    {
        "text": ["a diagram", "a dog", "two cats", "calico cat"],
        "image": image_to_base64(img_cat),
    },
]
response = endpoint.predict(instances=instances).predictions
response

Clean up resources:

In [ ]:
# Undeploy model and delete endpoint.
endpoint.delete(force=True)

# Delete models.
model.delete()

#### Image/text feature embedding

In [ ]:
model, endpoint = deploy_model(
    checkpoint=f"gs://{GCS_BUCKET}/{job_name}/checkpoints/epoch_latest.pt",
    model=model_name,
    precision=precision,
    task="feature-embedding",
)

NOTE: The model weights are downloaded after the deployment succeeds. An additional 5 minutes of waiting time is needed **after** the above model deployment step succeeds and before you run the next step below. Otherwise you might see a `ServiceUnavailable: 503 502:Bad Gateway` error when you send requests to the endpoint.

In [ ]:
import numpy as np

instances = [
    {
        "text": ["a diagram", "a dog", "a cat"],
        "image": image_to_base64(img_diagram),
    },
    {
        "image": image_to_base64(img_cat),
    },
    {
        "text": ["a diagram", "a dog", "two cats", "calico cat"],
    },
    {
        "text": "a single value",
    },
]
response = endpoint.predict(instances=instances).predictions
for pred in response:
    for k, v in pred.items():
        print(k, np.array(v).shape)

Clean up resources:

In [ ]:
# Undeploy model and delete endpoint.
endpoint.delete(force=True)

# Delete models.
model.delete()